In [5]:
# -*- coding: utf-8 -*-
import pandas as pd
from pathlib import Path
from pandas.tseries.offsets import MonthEnd

m_path = Path("宏观月度数据Verison20_仅时间行_清洗后_月末索引_abbr.xlsx")
q_path = Path("gdp_qoq.csv")

# 读月度：首行是列名，首列是日期
xl = pd.ExcelFile(m_path)
sheet = "growth" if "growth" in xl.sheet_names else ("我的数列" if "我的数列" in xl.sheet_names else xl.sheet_names[0])
m = pd.read_excel(m_path, sheet_name=sheet, header=0, index_col=0)

# 月末索引
m.index = pd.to_datetime(m.index, errors="coerce") + MonthEnd(0)
m = m.sort_index().asfreq("M")

# 按列转数值，跳过 datetime 列
for c in m.columns:
    if pd.api.types.is_datetime64_any_dtype(m[c]):
        # 真有日期型的误入列，直接丢弃或改名后保留原样二选一
        m = m.drop(columns=[c])
        continue
    if not pd.api.types.is_numeric_dtype(m[c]):
        s = m[c].astype(str).str.replace("%","", regex=False).str.replace(",","", regex=False)
        m[c] = pd.to_numeric(s, errors="coerce")

# 读季度
q = pd.read_csv(q_path)
if q.shape[1] == 1:
    q = pd.read_csv(q_path, index_col=0)
else:
    date_cols = [c for c in q.columns if str(c).lower() in ["date","time","period","quarter","季度","日期"]]
    q = pd.read_csv(q_path, index_col=(date_cols[0] if date_cols else 0))

# 季末月末索引
try:
    qi = pd.to_datetime(q.index, errors="raise")
    q.index = (qi.to_period("Q").to_timestamp("Q") + MonthEnd(0))
except Exception:
    q.index = pd.PeriodIndex(q.index.astype(str), freq="Q").to_timestamp("Q") + MonthEnd(0)

# 数值化 + 前缀

# 数值化
for c in q.columns:
    if not pd.api.types.is_numeric_dtype(q[c]):
        s = q[c].astype(str).str.replace("%","", regex=False).str.replace(",","", regex=False)
        q[c] = pd.to_numeric(s, errors="coerce")
q = q.sort_index()

# <<< 在这里添加 >>>
# 识别并重命名 GDP 列为 q_gdp_qoq / q_gdp_yoy / q_gdp
rename_map = {}
for c in q.columns:
    cl = str(c).lower()
    if any(k in cl for k in ["gdp", "rgdp", "国内生产总值"]):
        if any(k in cl for k in ["qoq", "环比"]):
            rename_map[c] = "q_gdp_qoq"
        elif any(k in cl for k in ["yoy", "同比"]):
            rename_map[c] = "q_gdp_yoy"
        else:
            rename_map[c] = "q_gdp"
q = q.rename(columns=rename_map)
if q.shape[1] == 1 and list(q.columns)[0] not in {"q_gdp_qoq","q_gdp_yoy","q_gdp"}:
    q.columns = ["q_gdp_qoq"]
# <<< 结束 >>>

# 统一前缀
q.columns = [c if str(c).startswith("q_") else f"q_{c}" for c in q.columns]


# 仅季末填入季度值
panel = m.copy()
q_on_m = q.reindex(panel.index)
q_on_m = q_on_m.where(panel.index.is_quarter_end[:, None])
panel = panel.join(q_on_m, how="outer")

out_path = m_path.with_name(m_path.stem + "_with_quarterly.xlsx")
with pd.ExcelWriter(out_path, engine="openpyxl") as w:
    panel.to_excel(w, sheet_name="monthly_panel")
    q.to_excel(w, sheet_name="quarterly_raw")

print(f"输出：{out_path}")


输出：宏观月度数据Verison20_仅时间行_清洗后_月末索引_abbr_with_quarterly.xlsx


C:\Users\kyx\AppData\Local\Temp\ipykernel_2664\3043281243.py:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  m = m.sort_index().asfreq("M")
